In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, BatchNormalization

Using TensorFlow backend.


## Loading Labels and Test Data

In [2]:
df_train = pd.read_csv('./Data/labels.csv')
df_test = pd.read_csv('./Data/sample_submission.csv')

OSError: File b'./Data/labels.csv' does not exist

In [ ]:
breed = set(df_train['breed'])
n_train = len(df_train)
n_breed = len(breed)

In [ ]:
target = pd.Series(df_train['breed'])
target_one_hot = pd.get_dummies(target, sparse=True)
one_hot_labels = np.asarray(target_one_hot)

In [ ]:
one_hot_labels

In [ ]:
im_size = 299
x_train = []
y_train = []
x_test = []

i=0
for filename, label in tqdm(df_train.values):
    img = cv2.imread('./Data/train/{}.jpg'.format(filename))
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(one_hot_labels[i])
    i+=1

In [ ]:
for filename in tqdm(df_test["id"].values):
    img = cv2.imread('./Data/test/{}.jpg'.format(filename))
    x_test.append(cv2.resize(img, (im_size, im_size)))

In [ ]:
x_train_raw = np.asarray(x_train, np.uint8) / 255
y_train_raw = np.asarray(y_train, np.float32)
x_test_raw = np.array(x_test, np.float32) / 255

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test_raw.shape)

In [ ]:
num_class = y_train_raw.shape[1]

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size = 0.3, random_state = 1)

In [ ]:
base_model = InceptionV3(include_top = False, weights = 'imagenet', input_tensor=None, input_shape=(im_size, im_size, 3), pooling=None, classes=1000)

In [ ]:
x =  base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid), verbose=1)

for layer in model.layers[:249]:
    layer.trainable = False

for layer in model.layers[249:]:
    layer.trainable = True

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1),
                  keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
                 ]

model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid), verbose=1)

In [ ]:
model.summary()
preds = model.predict(x_test, verbose=1)